## 4.1 評估機器學習模型
ML的目的是得到可以**泛化( generalize )**的模型，即在前所未見的數據上表現很好的模型，而過擬合則是核心難點，因此，本節將介紹如何評估機器學習模型，亦即，衡量模型泛化的能力。


### 4.1.1 訓練集、驗證集、測試集
我們通常在訓練集上訓練模型，在驗證數據上評估模型，一旦找到了最佳參數，就在測試數據上進行最後一次測試。<br><br>

而為了防止「訊息洩漏( information leak )」，我們不能基於測試集來調節模型，應該保留一個驗證集。因此，以下將介紹三種經典的評估方法：
* 簡單的切出驗證集數據<br>
![4.2.1](../img/4.2.1.PNG)
* K折驗證<br>
![4.2.1.1](../img/4.2.1.1.PNG)
* 帶有打亂數據的重複K折驗證



In [ ]:
# 4-1 簡單的切出驗證集數據
import numpy as np
from keras import *
data = np.arange(0,100000)                          # 產生raw_data
num_validation_samples = 10000
np.random.shuffle(data)                             # 通常要打亂數據

validation_data = data[:num_validation_samples]     # 定義驗證集
data = data[num_validation_samples:]

train_data = data[:]                                # 定義訓練集

model = get_model()                                 # |在訓練數據上
model.train(training_data)                          # |訓練模型，並
validation_score = model.evaluate(validation_data)  # |在驗證數據上
                                                    # |評估模型

# 因為有了val分數
# 現在我們可以調整模型(超參數)、重新訓練、再次評估、再次訓練、...
# 注意，我們只對模型做「一次」調整

model = get_model()                                 # |一旦調節好超參數，
model.train(np.concatenate([training_data,          # |通常就在所有非測試
                            validation_data]))      # |數據上從頭開始訓練
test_score = model.evaluate(test_data)              # |最終模型

###
# 缺點：可能因為輸入數據較少，
# 而驗證集以及測試集包含的樣本就少
# 從而無法在統計學上代表數據
###


In [ ]:
# 4-2 K折交叉驗證

###
# 如果模型性能的變化很大，
# 那麼這種方法很有用，
# 與前者一樣，這種方法也需要
# 獨立的驗證集進行模型校正。
###

k = 5                                       # 5折交叉驗證
num_validation_samples = len(data) // k

np.random.shuffle(data)

validation_scores = []
for fold in range(k):
    validation_data = data[num_validation_samples * fold:       # |選擇驗證數據分區 
                           num_validation_samples * (fold + 1)] # |
    training_data = data[:num_validation_samples * fold] + \
                    data[num_validation_samples * (fold + 1):]  # |使用剩餘數據作為訓練數據，
                                                                # |注意，+運算符是列表合併，
                                                                # |不是求和
    
    model = get_model()                                         # |創建一個全新的模型實例(未訓練)
    model.train(training_data)
    validation_score = model.evaluate(validation_data)
    validation_scores.append(validation_score)

validation_score = np.average(validation_scores)                # |最終驗證分數：
                                                                # |K折驗證分數的平均值

model = get_model()                                             # |在所有非測試數據上
model.train(data)                                               # |訓練最終模型
test_score = model.evaluate(test_data)                          # |


###
# 4-3 帶有打亂數據的重複K折驗證
# 適用於輸入數據相對較少，但又需要盡可能地評估模型
# 那麼我們可以選用打亂數據的重複K折驗證
# 作法為，多次使用K折驗證，在每次將數據化分為K個
# 分區之前都先將數據打亂，因此，這樣的計算代價很大。
###


### 4.1.2 評估模型的注意事項
* 數據代表性( data representation )：如果我們將數據前80%作為訓練集，剩餘20%作為測試集，這將導致訓練集中只包含類別0~7，測是類集中止包含類別8~9，這錯誤是很常見的!避免的做法為隨機打亂數據。
* 時間箭頭( the arrow of time )：若我們要根據過去預測未來(天氣、股票走勢等)，那麼我們在劃分數據時就不該打亂數據，因為這樣做會造成時間洩漏( temporal leak )：模型在未來數據上得到有效訓練，我們應該確保測試集中所有數據的時間都「晚於」訓練數據。
* 數據冗餘( redundancy in your data )：如果數據中的某些數據點出現了兩次(這在現實中的數據十分常見)，那麼打亂數據並劃分成訓練集與驗證集只會導致數據冗餘。因此，我們須確保訓練集與驗證集之間沒有交集。


